<a href="https://colab.research.google.com/github/Addofemi/Web-Scraping-Project/blob/main/Web_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
cd Web-Scraping-Project/

/content/Web-Scraping-Project


In [4]:
ls

README.md


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
BASE_URL = "http://books.toscrape.com/catalogue/page-{}.html"
BASE_SITE = "http://books.toscrape.com/catalogue/"

books = []

for page in range(1, 51):  # Loop through all 50 pages
    print(f"Scraping page {page}...")
    url = BASE_URL.format(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    for book in soup.find_all("article", class_="product_pod"):
        # Basic info from listing page
        title = book.h3.a["title"]
        price = book.find("p", class_="price_color").text.strip()
        rating = book.p["class"][1]

        # Book detail page link
        book_url = BASE_SITE + book.h3.a["href"].replace('../../../', '')
        book_resp = requests.get(book_url)
        book_soup = BeautifulSoup(book_resp.text, 'html.parser')

        # Category
        category = book_soup.find("ul", class_="breadcrumb").find_all("li")[2].text.strip()

        # Product description
        desc_tag = book_soup.find("div", id="product_description")
        description = desc_tag.find_next_sibling("p").text.strip() if desc_tag else ""

        # Table info (UPC, stock)
        table_rows = book_soup.find("table", class_="table table-striped").find_all("tr")
        upc = table_rows[0].find("td").text.strip()
        stock_text = table_rows[5].find("td").text.strip()

        # Extract stock number (e.g., "In stock (22 available)")
        stock_count = ''.join([c for c in stock_text if c.isdigit()])

        # Availability (from list page, cleaned)
        availability = book.find("p", class_="instock availability").text.strip()

        books.append({
            "Title": title,
            "Price": price,
            "Availability": availability,
            "Stock Count": stock_count,
            "Rating": rating,
            "Category": category,
            "UPC": upc,
            "Description": description
        })

        time.sleep(0.2)  # short delay for detail page requests

    time.sleep(1)  # delay for page requests

# Save to CSV
df = pd.DataFrame(books)
df.to_csv("../scraped_data/books_detailed_dataset.csv", index=False, encoding="utf-8")
print("Scraping complete! Data saved to books_detailed_dataset.csv")



In [2]:
!pwd

/content
